In [1]:
!pip install pandas
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
damage_train = "CrisisMMD_v2.0/crisismmd_datasplit_all/task_damage_text_img_train.tsv"
damage_dev = "CrisisMMD_v2.0/crisismmd_datasplit_all/task_damage_text_img_dev.tsv"
damage_test = "CrisisMMD_v2.0/crisismmd_datasplit_all/task_damage_text_img_test.tsv"


In [3]:
df_train = pd.read_csv(damage_train, sep='\t')
df_dev = pd.read_csv(damage_dev, sep='\t')
df_test = pd.read_csv(damage_test, sep='\t')


In [4]:
def remove_image_id_column(df):
    if "image_id" in df.columns:
        df = df.drop(columns=["image_id"])
    return df



In [5]:
df_train = remove_image_id_column(df_train)
df_dev = remove_image_id_column(df_dev)
df_test = remove_image_id_column(df_test)


In [6]:
df_train.head()
print("Train:", df_train.shape[0])
print("Dev:", df_dev.shape[0])
print("Test:", df_test.shape[0])



Train: 2468
Dev: 529
Test: 529


In [7]:
def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

df_train = remove_duplicates(df_train)
df_dev = remove_duplicates(df_dev)
df_test = remove_duplicates(df_test)

df_train.head()
print("Train:", df_train.shape[0])
print("Dev:", df_dev.shape[0])
print("Test:", df_test.shape[0])



Train: 2468
Dev: 529
Test: 529


In [8]:
import re

def remove_urls(text):
    return re.sub(r"http\S+|www\S+|t\.co\S+", "", text)

df_train["tweet_text"] = df_train["tweet_text"].apply(remove_urls)
df_dev["tweet_text"] = df_dev["tweet_text"].apply(remove_urls)
df_test["tweet_text"] = df_test["tweet_text"].apply(remove_urls)

df_train.sample(5)


,event_name,tweet_id,tweet_text,image,label
266,hurricane_harvey,909094133087608000,Hurricane Harvey Flooding: EPA to Close Housto...,data_image/hurricane_harvey/16_9_2017/90909413...,severe_damage
544,hurricane_maria,913892408621981000,Hurricane wiped out Puerto Rico's vibrant ag e...,data_image/hurricane_maria/29_9_2017/913892408...,mild_damage
2416,california_wildfires,918239396989669000,California's devastating wildfires and God via...,data_image/california_wildfires/11_10_2017/918...,severe_damage
1770,hurricane_irma,905695796422684000,Island hotel lashed by brutal wind and driving...,data_image/hurricane_irma/7_9_2017/90569579642...,mild_damage
730,california_wildfires,919734431052632000,Napa Fire Photo of the Week: Hell Descends on ...,data_image/california_wildfires/16_10_2017/919...,severe_damage


In [9]:
def remove_non_english(text):
    return re.sub(r"[^\x00-\x7F]+", "", text)  

df_train["tweet_text"] = df_train["tweet_text"].apply(remove_non_english)
df_dev["tweet_text"] = df_dev["tweet_text"].apply(remove_non_english)
df_test["tweet_text"] = df_test["tweet_text"].apply(remove_non_english)

df_train.sample(5)


,event_name,tweet_id,tweet_text,image,label
1798,mexico_earthquake,911701484852490000,Mexico earthquake: Searches could last weeks ...,data_image/mexico_earthquake/23_9_2017/9117014...,severe_damage
1534,mexico_earthquake,911833476780953000,Weird Breathing Road Seen In Aftermath Footage...,data_image/mexico_earthquake/24_9_2017/9118334...,severe_damage
309,california_wildfires,917923292148912000,Battle on to halt march of Northern California...,data_image/california_wildfires/11_10_2017/917...,severe_damage
1395,hurricane_harvey,907475708976525000,"Last week, I was on a boat! #Harveyflood #Harvey",data_image/hurricane_harvey/12_9_2017/90747570...,severe_damage
1948,hurricane_harvey,901809537828696000,"Rockport, TX gas station damaged by #Hurricane...",data_image/hurricane_harvey/27_8_2017/90180953...,severe_damage


In [10]:
df_train.to_csv('task_damage_text_img_train_cleaned.tsv', sep='\t', index=False)
df_dev.to_csv('task_damage_text_img_dev_cleaned.tsv', sep='\t', index=False)
df_test.to_csv('task_damage_text_img_test_cleaned.tsv', sep='\t', index=False)


In [11]:
df_humanitarian_train = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_humanitarian_text_img_train.tsv', sep='\t')
df_humanitarian_dev = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_humanitarian_text_img_dev.tsv', sep='\t')
df_humanitarian_test = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_humanitarian_text_img_test.tsv', sep='\t')


In [12]:
df_humanitarian_train = df_humanitarian_train.drop(columns=['image_id'])
df_humanitarian_dev = df_humanitarian_dev.drop(columns=['image_id'])
df_humanitarian_test = df_humanitarian_test.drop(columns=['image_id'])


In [13]:
df_train = df_humanitarian_train.drop_duplicates(subset='tweet_id')
df_dev = df_humanitarian_dev.drop_duplicates(subset='tweet_id')
df_test = df_humanitarian_test.drop_duplicates(subset='tweet_id')

In [14]:
import re

def clean_tweet_text(tweet):
    tweet = re.sub(r'http\S+|www\S+', '', tweet)
    tweet = re.sub(r'[^a-zA-Z\s\.,!?\'"]', '', tweet)
    return tweet

df_train['tweet_text'] = df_train['tweet_text'].apply(lambda x: clean_tweet_text(str(x)))
df_dev['tweet_text'] = df_dev['tweet_text'].apply(lambda x: clean_tweet_text(str(x)))
df_test['tweet_text'] = df_test['tweet_text'].apply(lambda x: clean_tweet_text(str(x)))

df_train = df_train[df_train['tweet_text'].str.strip() != '']
df_dev = df_dev[df_dev['tweet_text'].str.strip() != '']
df_test = df_test[df_test['tweet_text'].str.strip() != '']

df_train.to_csv('cleaned_humanitarian_train.csv', index=False)
df_dev.to_csv('cleaned_humanitarian_dev.csv', index=False)
df_test.to_csv('cleaned_humanitarian_test.csv', index=False)


/var/folders/3v/ctnck9xd4k17p0nrjgmcxp400000gn/T/ipykernel_23232/748638870.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['tweet_text'] = df_train['tweet_text'].apply(lambda x: clean_tweet_text(str(x)))


In [15]:
df_train.head()



,event_name,tweet_id,tweet_text,image,label,label_text,label_image,label_text_image
0,california_wildfires,917791291823591000,RT CalOES PLS SHARE Were capturing wildfire re...,data_image/california_wildfires/10_10_2017/917...,not_humanitarian,other_relevant_information,not_humanitarian,Negative
2,california_wildfires,917793137925459000,RT KAKEnews California wildfires destroy more ...,data_image/california_wildfires/10_10_2017/917...,infrastructure_and_utility_damage,infrastructure_and_utility_damage,infrastructure_and_utility_damage,Positive
5,california_wildfires,917815040962695000,RT TheAtlantic Photos of California's destruct...,data_image/california_wildfires/10_10_2017/917...,vehicle_damage,infrastructure_and_utility_damage,vehicle_damage,Negative
8,california_wildfires,917828283047260000,Why California's wildfires are worse in the fa...,data_image/california_wildfires/10_10_2017/917...,other_relevant_information,other_relevant_information,other_relevant_information,Positive
11,california_wildfires,917830092620836000,"Playing with my new friend, Chai. She's a Cali...",data_image/california_wildfires/10_10_2017/917...,not_humanitarian,not_humanitarian,not_humanitarian,Positive


In [16]:
df_dev.head()


,event_name,tweet_id,tweet_text,image,label,label_text,label_image,label_text_image
0,hurricane_maria,914509757070770000,Hurricane Maria Refugees Arrive in New Jersey ...,data_image/hurricane_maria/1_10_2017/914509757...,not_humanitarian,affected_individuals,not_humanitarian,Negative
1,srilanka_floods,870425128731107000,IFRC launches emergency appeal to help floodaf...,data_image/srilanka_floods/1_6_2017/8704251287...,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,not_humanitarian,Negative
2,hurricane_maria,912452432822702000,The administrations feeble response to Hurrica...,data_image/hurricane_maria/25_9_2017/912452432...,not_humanitarian,other_relevant_information,not_humanitarian,Negative
3,california_wildfires,920329866901381000,Family Miraculously Finds Dog They Lost When E...,data_image/california_wildfires/17_10_2017/920...,not_humanitarian,not_humanitarian,not_humanitarian,Positive
4,hurricane_irma,909744783115120000,Hurricane Irma on Twitter,data_image/hurricane_irma/18_9_2017/9097447831...,other_relevant_information,other_relevant_information,other_relevant_information,Positive


In [17]:
df_test.head()

,event_name,tweet_id,tweet_text,image,label,label_text,label_image,label_text_image
0,hurricane_harvey,905952332923338000,.Lendio has a great event tomorrow for both BY...,data_image/hurricane_harvey/8_9_2017/905952332...,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,Positive
1,mexico_earthquake,912022130396672000,Now ReposadoBar also Carlo surf band at pm pwy...,data_image/mexico_earthquake/24_9_2017/9120221...,not_humanitarian,not_humanitarian,not_humanitarian,Positive
2,mexico_earthquake,910700764808564000,"Earthquake in Mexico rocks PH Embassy, no Fili...",data_image/mexico_earthquake/21_9_2017/9107007...,infrastructure_and_utility_damage,other_relevant_information,infrastructure_and_utility_damage,Negative
3,hurricane_maria,927758858776465000,Guaynabo resident Efrain Diaz stands by a brid...,data_image/hurricane_maria/7_11_2017/927758858...,infrastructure_and_utility_damage,infrastructure_and_utility_damage,infrastructure_and_utility_damage,Positive
4,hurricane_harvey,907654309587779000,"After Harvey, Irma Leaves Its Own Trail of Des...",data_image/hurricane_harvey/12_9_2017/90765430...,other_relevant_information,other_relevant_information,other_relevant_information,Positive


In [18]:
import pandas as pd

df_informative_train = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_train.tsv', sep='\t')
df_informative_dev = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_dev.tsv', sep='\t')
df_informative_test = pd.read_csv('CrisisMMD_v2.0/crisismmd_datasplit_all/task_informative_text_img_test.tsv', sep='\t')

print(f"   - Before Cleaning: {df_informative_dev.shape[0]} rows, {df_informative_dev.shape[1]} columns")



   - Before Cleaning: 2237 rows, 9 columns


In [19]:
df_informative_train = df_informative_train.drop_duplicates(subset='tweet_id')
df_informative_dev = df_informative_dev.drop_duplicates(subset='tweet_id')
df_informative_test = df_informative_test.drop_duplicates(subset='tweet_id')


In [20]:
df_informative_train = df_informative_train[df_informative_train['label_text'] == 'informative']
df_informative_dev = df_informative_dev[df_informative_dev['label_text'] == 'informative']
df_informative_test = df_informative_test[df_informative_test['label_text'] == 'informative']


In [21]:
df_informative_train = df_informative_train.drop(columns=['label_image', 'image_id'])
df_informative_dev = df_informative_dev.drop(columns=['label_image', 'image_id'])
df_informative_test = df_informative_test.drop(columns=['label_image', 'image_id'])


In [22]:
import re

def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  #Remove URLS
    text = re.sub(r'@\S+', '', text)  #Remove mentions
    text = re.sub(r'[^\x00-\x7F]+', '', text) #Remove non-ASCII
    return text


df_informative_train.loc[:, 'tweet_text'] = df_informative_train['tweet_text'].apply(clean_text)
df_informative_dev.loc[:, 'tweet_text'] = df_informative_dev['tweet_text'].apply(clean_text)
df_informative_test.loc[:, 'tweet_text'] = df_informative_test['tweet_text'].apply(clean_text)


In [23]:
df_informative_train = df_informative_train.loc[:, ~df_informative_train.columns.str.contains('label')]
df_informative_dev = df_informative_dev.loc[:, ~df_informative_dev.columns.str.contains('label')]
df_informative_test = df_informative_test.loc[:, ~df_informative_test.columns.str.contains('label')]


In [24]:
df_informative_train.to_csv('cleaned_task_informative_text_img_train.tsv', sep='\t', index=False)
df_informative_dev.to_csv('cleaned_task_informative_text_img_dev.tsv', sep='\t', index=False)
df_informative_test.to_csv('cleaned_task_informative_text_img_test.tsv', sep='\t', index=False)


In [25]:
df_informative_train.head()

,event_name,tweet_id,tweet_text,image
0,california_wildfires,917791291823591425,RT PLS SHARE: Were capturing wildfire respons...,data_image/california_wildfires/10_10_2017/917...
2,california_wildfires,917793137925459968,RT California wildfires destroy more than 50 ...,data_image/california_wildfires/10_10_2017/917...
5,california_wildfires,917815040962695168,RT Photos of California's destructive wildfir...,data_image/california_wildfires/10_10_2017/917...
8,california_wildfires,917828283047260161,Why California's #wildfires are worse in the f...,data_image/california_wildfires/10_10_2017/917...
19,california_wildfires,917834588814155776,RT Deadly #California #wildfires force thousa...,data_image/california_wildfires/10_10_2017/917...


In [26]:
df_informative_dev.head()

,event_name,tweet_id,tweet_text,image
0,iraq_iran_earthquake,931465546129989632,#Iran #earthquake damage put at over 5bn pound...,data_image/iraq_iran_earthquake/17_11_2017/931...
1,hurricane_harvey,905064623199719425,We've lost track of how many houses/families ...,data_image/hurricane_harvey/5_9_2017/905064623...
2,hurricane_maria,922857566220283904,The gym above subway got destroyed. The cement...,data_image/hurricane_maria/24_10_2017/92285756...
3,hurricane_irma,909767231411769344,Famous Captive Orca Fends For Herself In Flori...,data_image/hurricane_irma/18_9_2017/9097672314...
4,california_wildfires,920571592131915777,Californias Sausalito Fire In Marin Captured I...,data_image/california_wildfires/18_10_2017/920...


In [27]:
df_informative_test.head()

,event_name,tweet_id,tweet_text,image
1,hurricane_harvey,906258753707790336,RT On our way to Warrior Restoration to help ...,data_image/hurricane_harvey/8_9_2017/906258753...
2,hurricane_maria,910542719864397824,Hurricane Maria batters Puerto Rico as a Cat 4...,data_image/hurricane_maria/20_9_2017/910542719...
3,hurricane_harvey,906219963152785408,ZZ Top Donating La Grange Download Sales To Hu...,data_image/hurricane_harvey/8_9_2017/906219963...
4,hurricane_maria,913009824195104768,8am #Maria update: holding steady as a strong ...,data_image/hurricane_maria/27_9_2017/913009824...
5,hurricane_maria,916053383383011328,. is hosting a Puerto Rico benefict concert to...,data_image/hurricane_maria/5_10_2017/916053383...


In [28]:
print(f"   - After Cleaning: {df_informative_dev.shape[0]} rows, {df_informative_dev.shape[1]} columns")


   - After Cleaning: 1612 rows, 4 columns


In [ ]:
unique_events_train = df_informative_train['event_name'].unique()
unique_events_dev = df_informative_dev['event_name'].unique()
unique_events_test = df_informative_test['event_name'].unique()

all_unique_events = set(unique_events_train) | set(unique_events_dev) | set(unique_events_test)

print("All Event Names:")
print(sorted(all_unique_events))    



All Event Names:
['california_wildfires', 'hurricane_harvey', 'hurricane_irma', 'hurricane_maria', 'iraq_iran_earthquake', 'mexico_earthquake', 'srilanka_floods']
